# **Text Classification with Spacy using [a news headline classification task](https://www.kaggle.com/datasets/rmisra/news-category-dataset)**

**News Category Dataset** is used to identify the type of news based on headlines and short descriptions. The two fields we’re interested in are category and headline. Our task will be to determine the category based on the text of the headline. Note that each headline in this set has a single category. In some classification tasks, multiple categories are allowed.

The headlines in this set fall into 41 categories, and you can see that the data is reasonably unbalanced, for that we’ll only use data from the top 4 categories: POLITICS, WELLNESS, ENTERTAINMENT and TRAVEL. There are 76,511 examples in total.


## Process the data


In [36]:
with open('resources\\News_Category_Dataset_v3.json') as f:
    lines = f.readlines()

In [37]:
import json

categories_count = {}

# Iterar a través de cada diccionario en la lista
for dic in lines:
    category = json.loads(dic)['category']
    
    # Actualizar el recuento de la categoría
    if category in categories_count:
        categories_count[category] += 1
    else:
        categories_count[category] = 1

sorted(categories_count.items(), key=lambda x: x[1], reverse=True)



[('POLITICS', 35602),
 ('WELLNESS', 17945),
 ('ENTERTAINMENT', 17362),
 ('TRAVEL', 9900),
 ('STYLE & BEAUTY', 9814),
 ('PARENTING', 8791),
 ('HEALTHY LIVING', 6694),
 ('QUEER VOICES', 6347),
 ('FOOD & DRINK', 6340),
 ('BUSINESS', 5992),
 ('COMEDY', 5400),
 ('SPORTS', 5077),
 ('BLACK VOICES', 4583),
 ('HOME & LIVING', 4320),
 ('PARENTS', 3955),
 ('THE WORLDPOST', 3664),
 ('WEDDINGS', 3653),
 ('WOMEN', 3572),
 ('CRIME', 3562),
 ('IMPACT', 3484),
 ('DIVORCE', 3426),
 ('WORLD NEWS', 3299),
 ('MEDIA', 2944),
 ('WEIRD NEWS', 2777),
 ('GREEN', 2622),
 ('WORLDPOST', 2579),
 ('RELIGION', 2577),
 ('STYLE', 2254),
 ('SCIENCE', 2206),
 ('TECH', 2104),
 ('TASTE', 2096),
 ('MONEY', 1756),
 ('ARTS', 1509),
 ('ENVIRONMENT', 1444),
 ('FIFTY', 1401),
 ('GOOD NEWS', 1398),
 ('U.S. NEWS', 1377),
 ('ARTS & CULTURE', 1339),
 ('COLLEGE', 1144),
 ('LATINO VOICES', 1130),
 ('CULTURE & ARTS', 1074),
 ('EDUCATION', 1014)]

In [39]:
from sklearn.model_selection import train_test_split
from spacy.tokens import DocBin
import spacy
import json

categories = ["POLITICS", "WELLNESS", "ENTERTAINMENT", "TRAVEL"]
headline = [
    json.loads(line)["headline"]
    for line in lines
    if json.loads(line)["category"] in categories
]
category = [
    json.loads(line)["category"]
    for line in lines
    if json.loads(line)["category"] in categories
]

X_train, X_test, y_train, y_test = train_test_split(
    headline, category, test_size=0.2, stratify=category, random_state=42
)


def convert(text_list: list, label_list: list, outfile: str):
    nlp = spacy.blank("en")
    db = DocBin()
    for text, label in zip(text_list, label_list):
        doc = nlp.make_doc(text)
        doc.cats = {category: 0 for category in categories}
        doc.cats[label] = 1
        db.add(doc)
    db.to_disk(outfile)


convert(X_train, y_train, "resources/4news_train.spacy")
convert(X_test, y_test, "resources/4_news_dev.spacy")


## Model training

In [41]:
!python -m spacy init config --pipeline textcat_multilabel 4news_config.cfg


⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: textcat_multilabel
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
4news_config.cfg
You can now add your data and train your pipeline:
python -m spacy train 4news_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
